In [20]:
import pandas as pd
import numpy as np

df = pd.read_csv('datasets/cbbAll.csv')

## Question 1

### Which conferences produce the best offenses and the best defenses each year?
 1. Which were the top five offensive and defensive conferences, year by year?
 2. Which were the top ten offensive and defensive conferences, out of all year by year results?
 3. Which were the top five offensive and defensive conferences cumulatively?

#### 1. Year by year performance

In [130]:
years = df['YEAR'].unique()
df_yby = df.groupby(['YEAR', 'CONF']).mean()

In [131]:
top_os = []
top_ds = []
a = [1,2,3,4]

for year in years:
    top_os.append(df_yby[df_yby.index.get_level_values('YEAR') == year]['ADJOE'].nlargest(5))
    top_ds.append(df_yby[df_yby.index.get_level_values('YEAR') == year]['ADJDE'].nsmallest(5))

In [132]:
[res for res in zip(top_os, top_ds)]

[(YEAR  CONF
  2013  B10     109.891667
        BE      107.740000
        ACC     107.466667
        P12     106.958333
        B12     106.880000
  Name: ADJOE, dtype: float64,
  YEAR  CONF
  2013  B10     92.250000
        BE      92.686667
        P12     94.550000
        ACC     95.383333
        SEC     95.564286
  Name: ADJDE, dtype: float64),
 (YEAR  CONF
  2014  B12     113.580000
        B10     112.575000
        BE      111.910000
        ACC     111.800000
        P12     110.983333
  Name: ADJOE, dtype: float64,
  YEAR  CONF
  2014  B10     96.275000
        ACC     97.886667
        SEC     98.207143
        B12     98.380000
        P12     98.533333
  Name: ADJDE, dtype: float64),
 (YEAR  CONF
  2015  ACC     111.033333
        BE      110.970000
        B10     110.514286
        B12     110.330000
        P12     108.233333
  Name: ADJOE, dtype: float64,
  YEAR  CONF
  2015  B12     93.560000
        ACC     94.826667
        BE      95.450000
        B10     95.514

#### 2. Top single season performances

In [133]:
df_yby['ADJOE'].nlargest(10)

YEAR  CONF
2018  BE      114.920000
2017  B12     114.810000
      ACC     114.560000
2018  B12     114.210000
2014  B12     113.580000
2016  ACC     113.506667
      B12     113.420000
2017  BE      112.790000
2014  B10     112.575000
2018  ACC     112.273333
Name: ADJOE, dtype: float64

In [134]:
df_yby['ADJDE'].nsmallest(10)

YEAR  CONF
2013  B10     92.250000
      BE      92.686667
2015  B12     93.560000
2019  B10     94.114286
      B12     94.380000
2013  P12     94.550000
2015  ACC     94.826667
2019  ACC     95.140000
2017  B12     95.230000
2013  ACC     95.383333
Name: ADJDE, dtype: float64

#### 3. Top cumulative performances

In [135]:
df_cum = df.groupby('CONF').mean()

In [136]:
print("Top Offenses:")
print(df_cum['ADJOE'].nlargest(5))
print("\n\nTop Defenses:")
print(df_cum['ADJDE'].nsmallest(5))

Top Offenses:
CONF
B12    112.184286
ACC    111.776471
BE     111.068000
B10    111.055319
SEC    109.404082
Name: ADJOE, dtype: float64


Top Defenses:
CONF
B12    95.584286
B10    95.886170
ACC    96.393137
BE     96.610667
SEC    96.762245
Name: ADJDE, dtype: float64


## Question 2

### Is a team with a better offense or a better defense more likely to have the most wins?
 1. What is the correlation between offensive and defensive efficiency and in-season win percentage (`PRE_PC`)?
 2. What is the correlation between offensive and defensive efficiency and tournament performance (`PFPC`)?

#### 1. Team efficiency vs win percentage

#### 2. Team efficiency vs postseason performance

## Question 3

### Is there a bias toward the "Major" conferences (ACC, Big East, Big Ten, Big 12, Pac-12, SEC) in how the postseason tournament is seeded?
 1. Do non-major teams on average need to have a higher power ranking than major teams in order to earn a top-4 seed?
 2. Do non-major teams of comparable power ranking to major teams achieve comparable postseason success?  

If the answer is Yes to both questions, it would indicate a bias in favor of the major teams, since they earn better seeds despite equal performance



In [137]:
majors = ['ACC', 'BE', 'B10', 'B12', 'P12', 'SEC']

#### 1. Average power rankings of top-4 seeds

In [139]:
df_maj_t4 = df[(df['CONF'].isin(majors)) & (df['SEED'].isin([1,2,3,4]))]
df_nmj_t4 = df[~(df['CONF'].isin(majors)) & (df['SEED'].isin([1,2,3,4]))]

In [142]:
df_maj_t4['BARTHAG'].mean()

0.9364464646464645

In [141]:
df_nmj_t4['BARTHAG'].mean()

0.9313461538461537

In [143]:
df_maj_t4['ADJOE'].mean() - df_nmj_t4['ADJOE'].mean()

2.1006216006215936

In [148]:
df_maj_t4['ADJDE'].mean() - df_nmj_t4['ADJDE'].mean()

1.4317793317793104

#### 2. Average postseason result of top-4 seeds

In [146]:
df_nmj_t4['PFPC'].mean()

0.9285076923076923

In [147]:
df_maj_t4['PFPC'].mean()

0.9441232323232321